This module covers Dynamic Partition Pruning (DPP), one of the most powerful optimizations in Spark 3.0 for Data Warehousing (Star Schema) workloads.

The Concept: "The Telepathic Scan"
In Spark 2.4, if you joined a massive Fact Table (partitioned by Date) with a small Dimension Table (filtered to "Yesterday"), Spark often scanned ALL partitions of the Fact Table because it didn't know which dates resulted from the join until it was too late.

In Spark 3.x, DPP allows the engine to:

Run the filter on the Dimension table first.

Take the resulting keys (e.g., specific dates).

Inject those keys directly into the Fact Table scanner.

Skip reading files that don't match the join keys.

Lab 1.2: Dynamic Partition Pruning
Objective: Prove that filtering a small table restricts the file scanning of a large joined table.

Step 1: Setup & Data Generation
We need a Partitioned Fact Table (Sales) and a Dimension Table (Stores). Run this cell to generate the data on MinIO.


In [1]:
# Cell 1: Initialize (Run Once)
# As soon as you run this, http://localhost:4040 will become live.
 

import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, rand, sum

# Initialize Spark with MinIO/S3 capabilities
# We use a 1GB limit to ensure we don't crash the container
spark = SparkSession.builder \
    .appName("Jupyter_DPP_Lesson") \
    .master("spark://spark-master:7077") \
    .config("spark.cores.max", "2") \
    .config("spark.executor.memory", "1g") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "password") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

# Reduce log noise
spark.sparkContext.setLogLevel("WARN")

print(f"✅ Spark Session Started!")
print(f"👉 Go open http://localhost:4040 in a new tab NOW.")
print(f"   (It will show 'No active stages', which is correct)")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-34904bf6-e91b-4687-aa10-c4ba699dc4b3;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 172ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 f

✅ Spark Session Started!
👉 Go open http://localhost:4040 in a new tab NOW.
   (It will show 'No active stages', which is correct)


In [9]:
#Cell 2: Generate Data (Run Once)
# This writes to S3. You will see jobs appear in the UI while this runs.


# Paths
FACT_PATH = "s3a://test-bucket/fact_sales"
DIM_PATH = "s3a://test-bucket/dim_stores"

print("🚀 Generating Data for DPP...")

# 1. Create Dimension Table (Regions)
# 5 Regions: 0, 1, 2, 3, 4
df_dim = spark.range(0, 5).select(col("id").alias("region_id"), (col("id") * 10).alias("store_code"))

print("\n\nDimensional data")
df_dim.show(truncate=False)
df_dim.write.format("delta").mode("overwrite").save(DIM_PATH)

# 2. Create Fact Table (Sales) - Partitioned by 'region_id'
# 5 Million rows distributed across 5 regions.
df_fact = spark.range(0, 5000000).select(
    col("id").alias("txn_id"), 
    (rand() * 100).alias("amount"),
    (col("id") % 5).alias("region_id") 
)

print("\n\nFact data")
df_fact.show(5, truncate=False)

print(f"💾 Writing Fact Table to {FACT_PATH}...")
df_fact.write.format("delta").partitionBy("region_id").mode("overwrite").save(FACT_PATH)

print("✅ Data Generated.")

🚀 Generating Data for DPP...


Dimensional data
+---------+----------+
|region_id|store_code|
+---------+----------+
|0        |0         |
|1        |10        |
|2        |20        |
|3        |30        |
|4        |40        |
+---------+----------+





Fact data
+------+------------------+---------+
|txn_id|amount            |region_id|
+------+------------------+---------+
|0     |89.91179044749333 |0        |
|1     |36.13195951313584 |1        |
|2     |62.93333165418805 |2        |
|3     |30.738846507147997|3        |
|4     |42.99228827796312 |4        |
+------+------------------+---------+
only showing top 5 rows

💾 Writing Fact Table to s3a://test-bucket/fact_sales...


✅ Data Generated.


### Observation: Data is generated in MiniO bucket


## Fact Sales Data
![Skew Stage Graph](screenshots/03_DPP_fact_sales_data.jpg)


## Dimension Region Data
![Skew Stage Graph](screenshots/04_DPP_dimension_region_data.jpg)

In [ ]:
Step 2: The Test (DPP Enabled vs Disabled)
We will join Sales (Fact) with Stores (Dim). We filter Stores to only look for Region 0.

Without DPP: Spark should scan all 5,000,000 rows (all 5 regions) from the Fact table, then filter them out during the join.

With DPP: Spark should only scan 1,000,000 rows (Region 0) from the Fact table.

In [10]:
# Cell 3: Test 1 - DPP DISABLED (The "Slow" Way)
# Run this, then look at the SQL tab in the UI.

print("--- RUNNING WITH DPP DISABLED ---")

# Disable DPP
spark.conf.set("spark.sql.optimizer.dynamicPartitionPruning.enabled", "false")
spark.catalog.clearCache() # Clear cache to force S3 read

start = time.time()

# Read Tables
fact = spark.read.format("delta").load(FACT_PATH)
dim = spark.read.format("delta").load(DIM_PATH)

# QUERY: Join Fact + Dim, filtering only on Dim
result = (
    fact.join(dim, "region_id")
    .filter(dim.store_code == 0) 
    .groupBy(dim.region_id)
    .agg(sum("amount"))
)

result.collect()
result.show(truncate=False)

print(f"⏱️ Duration: {time.time() - start:.2f} seconds")
print("👉 Check UI -> SQL Tab -> Click Query.")
print("   Look at the 'Scan parquet' box. It should read ALL files (approx 50MB).")

--- RUNNING WITH DPP DISABLED ---
+---------+-------------------+
|region_id|sum(amount)        |
+---------+-------------------+
|0        |5.002179944479497E7|
+---------+-------------------+

⏱️ Duration: 1.58 seconds
👉 Check UI -> SQL Tab -> Click Query.
   Look at the 'Scan delta' box. It should read ALL files (approx 50MB).


## DPP disabled reads all data - 10 files 57 MB
![Skew Stage Graph](screenshots/05_DPP_disabled_read_all_rows.jpg)

In [11]:
# Cell 4: Test 2 - DPP ENABLED (The "Fast" Way)
# Run this immediately after. The session stays alive.



print("--- RUNNING WITH DPP ENABLED ---")

# Enable DPP
spark.conf.set("spark.sql.optimizer.dynamicPartitionPruning.enabled", "true")
spark.catalog.clearCache()

start = time.time()

fact = spark.read.format("delta").load(FACT_PATH)
dim = spark.read.format("delta").load(DIM_PATH)

# We filter on 'store_code', NOT the join key 'region_id'.
# Spark CANNOT statically optimize this. It MUST use DPP.
result = (
    fact.join(dim, "region_id")
    .filter(dim.store_code == 0) 
    .groupBy(dim.region_id)
    .agg(sum("amount"))
)

result.collect()
result.show(truncate=False)

print(f"⏱️ Duration: {time.time() - start:.2f} seconds")
print("👉 Check UI -> SQL Tab -> Click the NEW Query.")
print("   Look at 'Scan parquet'. It should read ONLY 1/5th of the files (approx 10MB).")


--- RUNNING WITH DPP ENABLED ---
+---------+--------------------+
|region_id|sum(amount)         |
+---------+--------------------+
|0        |5.0021799444798514E7|
+---------+--------------------+

⏱️ Duration: 1.20 seconds
👉 Check UI -> SQL Tab -> Click the NEW Query.
   Look at 'Scan delta'. It should read ONLY 1/5th of the files (approx 10MB).


## DPP disabled reads all data - 10 files 57 MB
![Skew Stage Graph](screenshots/06_DPP_enabled_reads_less_rows.jpg)

How to verify in the UI
Since you won't be rushing against a spark.stop() timer, you can take your time:

Go to the SQL / DataFrame tab.

You will see two completed queries.

Click the Description of the first one (DPP Disabled).

Find the Scan Delta node.

Hover over it. Look for "size of files read". It should be the total size of your dataset.

Click the Description of the second one (DPP Enabled).

Find the Scan Delta node.

The "size of files read" should be significantly smaller (exactly 20% of the total, since we selected 1 out of 5 regions).

This confirms that Spark "pushed" the filter from the small table into the storage reader of the big table before reading the files.

# 🧠 Concept: How Dynamic Partition Pruning (DPP) Actually Works

**The Question:** Is DPP based on *Static File Metadata* (like Parquet footers) or *Dynamic Engine Logic*?

**The Answer:** It is **Dynamic Engine Logic**.

While Parquet/Delta do store metrics (min/max values) in their footers for "Data Skipping" (skipping row groups *inside* a file), **Dynamic Partition Pruning (DPP)** operates at a higher level: it skips entire **Partition Directories** based on the results of a running query.

---

### 🔄 The Mechanics: "The Feedback Loop"

Usually, a database reads tables starting from the bottom (Scan) and moves up (Filter -> Join). DPP breaks this rule by creating a **Feedback Loop**.



Here is the step-by-step execution flow:

#### 1. The Dimension Side (The "Scout")
Spark looks at your query plan and notices a specific pattern:
* **Table A (Fact):** Partitioned by `region_id`.
* **Table B (Dim):** Filtered (e.g., `store_code == 0`) and joined on `region_id`.

Spark decides: *"I need to run the Dimension side first."*
It runs the filter on the Dimension table (`dim_stores`) and finds the matching keys.
* *Result:* `region_id = [0]`

#### 2. The Broadcast (The "Phone Call")
This is the magic part. Instead of just sending that result to the Join operator, Spark wraps those keys (`[0]`) into a **BroadcastExchange**.
It essentially "broadcasts" these valid keys to all executors.

#### 3. The Injection (The "Guard at the Door")
Spark takes that Broadcast variable and **injects it as a Filter** directly into the **Fact Table Scanner**.

Before the Fact Table reader even looks at the file system, it checks this dynamic list.
* *Scanner:* "I see folders for Regions 0, 1, 2, 3, 4. Which ones should I open?"
* *DPP Filter:* "I just got a call from the Dimension table. Only open **Region 0**."

#### 4. The Pruned Scan
The scanner now ignores the directories for Regions 1, 2, 3, and 4. It only reads the files in `region_id=0`.

---

### 🆚 Visualizing the Difference: Parquet Metrics vs. DPP

It is crucial to distinguish between these two optimizations:

| Feature | **Data Skipping (Parquet/Delta Metrics)** | **Dynamic Partition Pruning (DPP)** |
| :--- | :--- | :--- |
| **What it uses** | **Static Metadata:** Min/Max statistics stored in the file footer or `_delta_log`. | **Runtime Results:** The actual output of a query running on another table. |
| **Scope** | **Intra-Partition:** "Does this specific *file* contain ID 500?" | **Inter-Partition:** "Does this whole *folder* contain the Joined Keys?" |
| **Trigger** | `WHERE id = 500` (Direct Filter) | `JOIN ... ON ...` (Indirect Filter) |